*¿Qué es un dask bag?*

Dask Bag implementa operaciones como map, filter, fold y groupby en colecciones de objetos genéricos de Python.

Hace esto en paralelo con una pequeña huella de memoria usando iteradores de Python.

Es similar a una versión paralela de PyToolz o una versión Pythonic de PySpark RDD.

Las bolsas Dask coordinan muchas listas de Python o iteradores, cada uno de los cuales forma una partición de una colección más grande.

#### Operaciones dask bag

In [33]:
import dask.bag as db
import json

b = db.from_sequence([1,2,3,4,5,6,7,8,9,10], npartitions= 4)
b

dask.bag<from_sequence, npartitions=4>

In [39]:
b.map(lambda x : x**2).compute()

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [38]:
b.filter(lambda x: x % 2 == 0).map(lambda x: x == 2).sum().compute()

1

¿Como trabajar con dask bag y json?

In [145]:
text_bag = db.read_text("data/politicians/*.json")

dict_bag = text_bag.map(json.loads)

print(dict_bag.take(1))

({'gender': 'male', 'id': '0067809f-2146-46ee-9249-a0d1be5ecb4f', 'identifiers': [{'identifier': '1771', 'scheme': 'everypolitician_legacy'}], 'image': 'http://www.wolesi.website/Media/Images/mine/m-abduljabbar.jpg', 'images': [{'url': 'http://www.wolesi.website/Media/Images/mine/m-abduljabbar.jpg'}], 'name': 'Haji Abdul Jabar , Takhar'},)


Sabemos que algunos diccionarios tienen la llave birth_date.

¿Cuantos registros tienen esta llave birth_date?

In [146]:
print(dict_bag.count().compute())

def has_birth_date(dictionary):
  return 'birth_date' in dictionary

filtered_bag = dict_bag.filter(has_birth_date)

print(filtered_bag.count().compute())

3860
3120


¿Cual será el minímo, máximo y promedio de años de nacimiento?

In [147]:
filtered_bag.pluck('birth_date').take(15) #Selección de la llave 'birth_date'

('1972',
 '1962',
 '1961',
 '1954',
 '1979',
 '1969',
 '1981-04-19',
 '1974',
 '1975',
 '1965',
 '1957',
 '1985-03-22',
 '1962',
 '1964',
 '1946')

In [148]:
birth_date_bag = filtered_bag.pluck('birth_date')

# Extae el año como un número entero de la cadenas de fecha de nacimiento
birth_year_bag = birth_date_bag.map(lambda x: int(x[:4])) #1985-03-22 => tomará 1985

min_year = birth_year_bag.min()
max_year = birth_year_bag.max()
mean_year = birth_year_bag.mean()

print(dask.compute(min_year, max_year, mean_year))

(1886, 1995, 1957.8483974358974)


**Convertir datos no estructurados en DataFrames**

Puede ser complicado trabajar con map, filter en objetos Json.

Para esto podemos convertir en marcos de datos DataFrame.

In [107]:
dict_bag.take(1)

({'gender': 'male',
  'id': '0067809f-2146-46ee-9249-a0d1be5ecb4f',
  'identifiers': [{'identifier': '1771', 'scheme': 'everypolitician_legacy'}],
  'image': 'http://www.wolesi.website/Media/Images/mine/m-abduljabbar.jpg',
  'images': [{'url': 'http://www.wolesi.website/Media/Images/mine/m-abduljabbar.jpg'}],
  'name': 'Haji Abdul Jabar , Takhar'},)

In [115]:
df_bag = dict_bag.to_dataframe()
df_bag.head(3)

,gender,id,identifiers,image,images,name
0,male,0067809f-2146-46ee-9249-a0d1be5ecb4f,"[{'identifier': '1771', 'scheme': 'everypoliti...",http://www.wolesi.website/Media/Images/mine/m-...,[{'url': 'http://www.wolesi.website/Media/Imag...,"Haji Abdul Jabar , Takhar"
1,female,0282b3d6-c7a4-47ab-bcbe-cdc3e2b26437,"[{'identifier': '1936', 'scheme': 'everypoliti...",http://www.wolesi.website/Media/Images/mine/sh...,[{'url': 'http://www.wolesi.website/Media/Imag...,Alhaj Shakila Hashimi-Logar
2,female,030b25fb-05d6-4acb-809d-c122db9e0f94,"[{'identifier': '1847', 'scheme': 'everypoliti...",http://www.wolesi.website/Media/Images/mine/sh...,[{'url': 'http://www.wolesi.website/Media/Imag...,Shukria Barakzai-Kabul


**Reestructuración de un diccionario**

Ahora desea limpiar los datos de los políticos y moverlos a un Dask DataFrame.

Sin embargo, los datos de los políticos están anidados, por lo que deberá procesarlos un poco más antes de que encajen en un DataFrame.

Una parte particular de los datos que desea extraer está enterrada en algunas capas dentro del diccionario.

In [135]:
def extract_images_url(x):
    x['images_url'] = x['images'][0]['url'] #Selección de la llave images valor 0 y url.
    return x
  
# Función que corre para cada elemento del dask bak.
dict_bag = dict_bag.map(extract_url)

print(dict_bag.take(1))

({'gender': 'male', 'id': '0067809f-2146-46ee-9249-a0d1be5ecb4f', 'identifiers': [{'identifier': '1771', 'scheme': 'everypolitician_legacy'}], 'image': 'http://www.wolesi.website/Media/Images/mine/m-abduljabbar.jpg', 'images': [{'url': 'http://www.wolesi.website/Media/Images/mine/m-abduljabbar.jpg'}], 'name': 'Haji Abdul Jabar , Takhar', 'url': 'http://www.wolesi.website/Media/Images/mine/m-abduljabbar.jpg', 'images_url': 'http://www.wolesi.website/Media/Images/mine/m-abduljabbar.jpg'},)


**Conversión a marco de datos**

Desea hacer un DataFrame a partir de los datos JSON del político.

Ahora que ha anidado los datos, todo lo que necesita hacer es seleccionar las claves para 
mantenerlas como columnas en el DataFrame.

In [154]:
def select_keys(dictionary, keys_to_keep):
  new_dict = {}
  # Loop through kept keys and add them to new dictionary
  for k in keys_to_keep:
    new_dict[k] = dictionary[k]
  return new_dict

# Use the select_keys to reduce to the 4 required keys
filtered_bag = dict_bag.map(select_keys, keys_to_keep=['images'])

# Convert the restructured bag to a DataFrame
df = filtered_bag.to_dataframe()

# Print the first few rows of the DataFrame
print(df.head())

                                              images
0  [{'url': 'http://www.wolesi.website/Media/Imag...
1  [{'url': 'http://www.wolesi.website/Media/Imag...
2  [{'url': 'http://www.wolesi.website/Media/Imag...
3  [{'url': 'http://www.wolesi.website/Media/Imag...
4  [{'url': 'http://www.wolesi.website/Media/Imag...


**Usando cualquier datos en dask bags.**

Es posible trabajar con archivos .mp4 y almacenar los registros en dask bag.